[PlayTypes](https://statsapi.web.nhl.com/api/v1/playTypes)
[Example game](https://statsapi.web.nhl.com/api/v1/game/2017020001/feed/live)
[Example team](https://statsapi.web.nhl.com/api/v1/teams/10)

# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
if pathlib.Path().resolve().name == 'notebooks':
    %cd ..
%pwd

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from src.data.load import NHLDataDownloader
from scipy import stats
from src.utils import normalize


In [ ]:
# Utilisation du téléchargeur de données NHL
nhl_2019 = NHLDataDownloader()
df_2019= nhl_2019.load_df_shots(2019)

In [ ]:
df_2019.head(10)

In [ ]:
ddf_2019 = df_2019[df_2019['Type'] != '']
df_g_2019 = ddf_2019.groupby(['Type', 'Goal']).size().to_frame('Counts').reset_index()
df_g_2019['Percentage'] = df_g_2019['Counts'] / df_g_2019.groupby('Type')['Counts'].transform('sum') * 100

px.bar(df_g_2019,
       x='Type',
       y='Counts',
       color='Goal',
       title=f"Number of shots, by distance - Season {season_year}",
       log_y=True,
       text=df_g_2019['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
       height=550
       ).update_xaxes(categoryorder='total descending')

In [ ]:
# Utilisation du téléchargeur de données NHL
nhl_2020 = NHLDataDownloader()
# Étape 1: Spécifier la saison
season_year = 2020
nhl_2020.set_season(season_year)
df_2020= nhl_2020.load_df_shots()
df_2020

In [ ]:
ddf_2020 = df_2020[df_2020['Type'] != '']
df_g_2020 = ddf_2020.groupby(['Type', 'Goal']).size().to_frame('Counts').reset_index()
df_g_2020['Percentage'] = df_g_2020['Counts'] / df_g_2020.groupby('Type')['Counts'].transform('sum') * 100

px.bar(df_g_2020,
       x='Type',
       y='Counts',
       color='Goal',
       title=f"Number of shots, by distance - Season {season_year}",
       log_y=True,
       text=df_g_2020['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
       height=550
       ).update_xaxes(categoryorder='total descending')

In [ ]:
df_g = df.groupby(['Goal','Net_distance']).size().to_frame('Counts').reset_index()
df_g['Bins'] = pd.cut(df_g['Net_distance'], 12)
df_g = df_g.groupby(['Goal','Bins'], observed=True)['Counts'].sum().to_frame('Counts').reset_index()
df_g['Percentage'] = df_g['Counts'] / df_g.groupby('Bins', observed=True)['Counts'].transform('sum') * 100
df_g['Bins'] = df_g['Bins'].astype('str')

px.bar(df_g,
       x='Bins',
       y='Counts',
       color='Goal',
       title=f"Number of shots, by type - Season {season_year}",
       log_y=True,
       text=df_g['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
       height=550
       )


# Raw data examples


In [ ]:
# Utilisation du téléchargeur de données NHL

# Étape 1: Spécifier la saison
season_year = 2017
nhl_downloader = NHLDataDownloader(season_year)

# Étape 2: Télécharger les données
season_data = nhl_downloader.load_data()


In [ ]:
for play in season_data['regulars'][0]['liveData']['plays']['allPlays']:
  if play['result']['event'] == 'Shot':
    shot_exemple = play
  if play['result']['event'] == 'Goal':
    goal_exemple = play

shot_exemple

In [ ]:
goal_exemple

# Data interpretation

In [ ]:
season = nhl_downloader.load_processed_data()

In [ ]:
# What are the strengh types ?
strengh_types = set()
for game in season.regulars:
  for play in game.plays:
    if play.result.strength:
      strengh_types.add(play.result.strength)

strengh_types

In [ ]:
# What are the shot types ?

shot_types = set()
for game in season.regulars:
  for play in game.plays:
    res = play.result
    if (res.event == 'Goal' or res.event == 'Shot') and res.secondaryType:
      shot_types.add(res.secondaryType)

shot_types

# To DataFrames

In [ ]:
df = nhl_downloader.load_df_shots()
df

# Visualisations simples

## Number of shots, by type

In [ ]:
ddf = df[df['Type'] != '']
df_g = ddf.groupby(['Type', 'Goal']).size().to_frame('Counts').reset_index()
df_g['Percentage'] = df_g['Counts'] / df_g.groupby('Type')['Counts'].transform('sum') * 100

px.bar(df_g,
       x='Type',
       y='Counts',
       color='Goal',
       title="Number of shots, by type",
       log_y=True,
       text=df_g['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
       height=550
       ).update_xaxes(categoryorder='total descending')

In [ ]:
df_g = df.groupby(['Goal','Net_distance']).size().to_frame('Counts').reset_index()
df_g['Bins'] = pd.cut(df_g['Net_distance'], 12)
df_g = df_g.groupby(['Goal','Bins'], observed=True)['Counts'].sum().to_frame('Counts').reset_index()
df_g['Percentage'] = df_g['Counts'] / df_g.groupby('Bins', observed=True)['Counts'].transform('sum') * 100
df_g['Bins'] = df_g['Bins'].astype('str')

px.bar(df_g,
       x='Bins',
       y='Counts',
       color='Goal',
       title="Number of shots, by distance",
       log_y=True,
       text=df_g['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
       height=550,
       labels=dict(Bins='Distance (ft)')
       )

## Distance to net

Nets are positioned at x = -89,89 and y = 0, the teams switch sides at each periods.

In [ ]:
df_g = df.groupby(['Goal','Net_distance']).size().to_frame('Counts').reset_index()
df_g['Bins'] = pd.cut(df_g['Net_distance'], 12)
df_g = df_g.groupby(['Goal','Bins'], observed=True)['Counts'].sum().to_frame('Counts').reset_index()
df_g['Percentage'] = df_g['Counts'] / df_g.groupby('Bins', observed=True)['Counts'].transform('sum') * 100
df_g['Bins'] = df_g['Bins'].astype('str')

px.bar(df_g,
       x='Bins',
       y='Counts',
       color='Goal',
       title="Number of shots, by distance",
       log_y=True,
       text=df_g['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
       height=550
       )

## Percentage of Goals by type and distance

In [ ]:
df_td = df.copy()
df_td = df_td[(df_td['Type'] != '') & (~df_td['Net_distance'].isna())]
df_td['Distance'] = pd.qcut(df_td['Net_distance'], 12)

grouped = df_td.groupby(['Type', 'Distance']).agg(Total_Shots=('Goal', 'size'), Goals=('Goal', 'sum')).reset_index()
grouped['Percentage'] = grouped['Goals'] / grouped['Total_Shots'] * 100

q3 = grouped.Percentage.quantile(0.75)
iqr = q3 - grouped.Percentage.quantile(0.25)
upper_fence = q3 + 1.5 * iqr
# Emperical value
upper_fence = 30
grouped = grouped[grouped['Percentage'] < upper_fence]

grouped = grouped.pivot(index='Type', columns='Distance')['Percentage']
grouped.columns = grouped.columns.astype('str')
grouped.fillna(0, inplace=True)

fig = px.imshow(grouped, width=800, height=650, labels=dict(y='Shot Type', x="Distance (ft)", color="Goal Percentage"),
                title='Goals success rate by type and distance')
fig.update_xaxes(side="top")
fig.update_layout(title_font_size=30)
fig.show()

In [ ]:
df_sl = df[df.X_dist < 90]
df_sl = df_sl.groupby(['X_dist', 'Y']).size().to_frame('Counts').reset_index()
df_sl = df_sl.pivot(index='X_dist', columns='Y')['Counts'].fillna(0)

fig = px.imshow(np.log(df_sl + 1), width=600, height=600,
          labels=dict(x='Distance from center of rink (ft)', y="Distance from goal line (ft)", color="Number of shots (log)"),
          title='Shots location',
          )
fig.update_layout(title_font_size=30)

## Outil de débogage interactif

In [ ]:
game = season.regulars[1]
df = game.to_df()
match_title = f'{game.home_team.name} VS {game.away_team.name}'

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="x", y="y", animation_frame="event idx", range_x=[-100,100], range_y=[-42.5,42.5])

fig.update_traces(mode='markers',
                         marker_size=15,
                         marker_color="#111111")

fig.add_layout_image(
  source="https://raw.githubusercontent.com/udem-ift6758/project-template/main/figures/nhl_rink.png",
  xref="x",
  yref="y",
  x=-100,
  y=42.5,
  sizex=200,
  sizey=85,
  sizing="stretch",
  opacity=0.9,
  layer="below"
)
fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[-100, 100])
fig.update_yaxes(
    showline=False,
    zeroline=False,
    showgrid=False,
    range=[-42.5, 42.5],
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.update_layout(
  title={
        'text': match_title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
  autosize=False,
  template="plotly_white",)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["redraw"] = True

for id, fr in enumerate(fig.frames):
  fr.layout.title = df.loc[id]['description']

for step in fig.layout.sliders[0].steps:
    step["args"][1]["frame"]["redraw"] = True


fig.show()

In [ ]:
fig.write_html("plotly_demo_3.html", auto_play = False)

In [ ]:
# Create a function to update the Plotly scatter plot based on the selected game
from ipywidgets import widgets, interact, IntSlider

def update_plot(game_idx):
    '''if season_type == "Regular":
        game = season.regulars[game_idx]
    elif season_type == "Playoff":
        game = season.playoffs[game_idx]'''

    game = season.regulars[game_idx]
    df = game.to_df()
    match_title = f'{game.home_team.name} VS {game.away_team.name}'

    fig = px.scatter(df, x="x", y="y", animation_frame="event idx", range_x=[-100,100], range_y=[-42.5,42.5])

    fig.update_traces(mode='markers',
                             marker_size=15,
                             marker_color="#111111")

    fig.add_layout_image(
      source="https://raw.githubusercontent.com/udem-ift6758/project-template/main/figures/nhl_rink.png",
      xref="x",
      yref="y",
      x=-100,
      y=42.5,
      sizex=200,
      sizey=85,
      sizing="stretch",
      opacity=0.9,
      layer="below"
    )
    fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[-100, 100])
    fig.update_yaxes(
        showline=False,
        zeroline=False,
        showgrid=False,
        range=[-42.5, 42.5],
        scaleanchor = "x",
        scaleratio = 1,
      )

    fig.update_layout(
      title={
            'text': match_title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
      autosize=False,
      template="plotly_white",)

    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["redraw"] = True

    for id, fr in enumerate(fig.frames):
      fr.layout.title = df.loc[id]['description']

    for step in fig.layout.sliders[0].steps:
        step["args"][1]["frame"]["redraw"] = True

    fig.show()

# Create a dropdown widget to select the match
match_dropdown = widgets.Dropdown(options=range(len(season.regulars)), description="Select Match:")


# Create an interactive output that displays the plot
interactive_output = widgets.interactive_output(update_plot, {'game_idx': match_dropdown})

# Display the widgets
widgets.VBox([match_dropdown, interactive_output])


# Visualisations avancées

In [ ]:
nhl_2018 = NHLDataDownloader()

nhl_2018.set_season(2018)
df_2018 = nhl_2018.load_df_shots()

In [ ]:
def get_shots_location(df: pd.DataFrame, team: str) -> pd.DataFrame:
    """
    Get the difference between the average number of shot by all teams vs the specified team.

    Args:
        df : DataFrame returned by the fonction load_df_shots.
        team: Tricode of the team. e.g. 'MTL'
    """
    df_tot = df[df.X_dist < 90]
    df_team = df_tot[df_tot.Team == team]
    ng=df_team.Game_id.nunique()
    #print(ng)

    # Get the average number of shots for every location
    df_tot = df_tot.groupby(['X_dist', 'Y']).size().to_frame('Counts').reset_index()
    df_tot = df_tot.pivot(index='X_dist', columns='Y')['Counts'].fillna(0)
    df_tot = df_tot / (df.Game_id.nunique() * 2)
    df_tot = pd.melt(df_tot.reset_index(), id_vars='X_dist', value_vars=df_tot.columns)

    # Get the team average number of shots for every location
    df_team = df_team.groupby(['X_dist', 'Y']).size().to_frame('Counts').reset_index()
    df_team = df_team.pivot(index='X_dist', columns='Y')['Counts'].fillna(0)
    df_team = df_team / ng

    df_team = pd.melt(df_team.reset_index(), id_vars='X_dist', value_vars=df_team.columns)

    # Compute the difference
    df_sl = pd.merge(df_tot, df_team, on=["X_dist", "Y"],  how="outer").fillna(0)
    df_sl["dif"] = df_sl.value_y - df_sl.value_x


    # Compute the binned difference
    df_sl['ybin'] = pd.cut(df_sl.Y, 15)
    df_sl['xbin'] = pd.cut(df_sl.X_dist, 15)
    df_sl['dif_bin'] = df_sl.groupby(['xbin', 'ybin'], observed=True).dif.transform('sum')

    return df_sl

In [ ]:
def get_dens(team,df):
    """
    Compute the difference between shots after applying kde.

    Args:
        team: Tricode of the team. e.g. 'MTL'
    """

    dfs = get_shots_location(df, team)
    max = dfs.dif_bin.max()

    values = [dfs.X_dist.values, dfs.Y.values]
    weights_tot = dfs.value_x.values
    weights_team = dfs.value_y.values

    gauss_tot = stats.gaussian_kde(values, bw_method=0.3,weights=weights_tot)
    gauss_team = stats.gaussian_kde(values, bw_method=0.3,weights=weights_team)


    x = np.linspace(0, 89, 90)
    y = np.linspace(-42, 42, 85)
    X, Y = np.meshgrid(x, y)
    xy = np.vstack([X.ravel(), Y.ravel()])
    dens_tot = np.exp(gauss_tot(xy)).reshape((X.shape))
    dens_team= np.exp(gauss_team(xy)).reshape((X.shape))

    dens = dens_team - dens_tot

    ratio = np.array(dens).reshape(-1).min() / np.array(dens).reshape(-1).max()
    return normalize(np.flip(dens), max * ratio, max)

In [ ]:
from PIL import Image

pyLogo = Image.open("figures/half_nhl_rink.png")

fig = go.Figure()

x = np.linspace(0, 89, 90)
y = np.linspace(-42, 42, 85)

teams = df.Team.unique()
visibility = [False] * len(teams)
dropdown_list = []

fig.add_trace(
    go.Contour(
        z=get_dens(teams[0],df),
        # z=np.flip(dens_team - dens_tot),
        x=x,
        y=y,
        opacity = 0.6,
        colorscale='RdBu_r',
        contours_coloring='fill',  # This will fill the contours
        contours=dict(start=-0.5, end=0.5, size=0.05),  # You can adjust these parameters
        visible=True
    )
)

vis = visibility.copy()
vis[0] = True

dropdown_list.append(dict(
                args=[{"visible": vis}],
                label=teams[0],
                method="restyle"
            ))

for id, team in enumerate(teams[1:]):


# Plotly Contour Plot
    fig.add_trace(
        go.Contour(
            z=get_dens(team,df),
            x=x,
            y=y,
            opacity = 0.6,
            colorscale='RdBu_r',
            contours_coloring='fill',
            contours=dict(start=-0.5, end=0.5, size=0.05),
            visible=False
        )
    )

    vis = visibility.copy()
    vis[id + 1] = True

    dropdown_list.append(dict(
                    args=[{"visible": vis}],
                    label=team,
                    method="restyle"
                ))


fig.update_layout(
    height=600,
    width=680,
    title = f'Season {season_year}',
    title_font_size=30,
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_list,
            showactive=True,
            x=0.35,
            xanchor="left",
            y=1.18,
            yanchor="top"
        ),
    ]
)
fig.add_layout_image(
        dict(
            source=pyLogo,
            xref="x",
            yref="y",
            x=0,
            y=42.5,
            sizex=100,
            sizey=85,
            opacity=1,
            layer="below")
)

fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[0, 100])
fig.update_yaxes(
    showline=False,
    zeroline=False,
    showgrid=False,
    range=[-42.5, 42.5],
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

In [ ]:
from PIL import Image
nhl_2018 = NHLDataDownloader()

nhl_2018.set_season(2018)
df_2018 = nhl_2018.load_df_shots()
pyLogo = Image.open("figures/half_nhl_rink.png")

fig = go.Figure()

x = np.linspace(0, 89, 90)
y = np.linspace(-42, 42, 85)

teams = df_2018.Team.unique()
visibility = [False] * len(teams)
dropdown_list = []

fig.add_trace(
    go.Contour(
        z=get_dens(teams[0],df_2018),
        # z=np.flip(dens_team - dens_tot),
        x=x,
        y=y,
        opacity = 0.6,
        colorscale='RdBu_r',
        contours_coloring='fill',  # This will fill the contours
        contours=dict(start=-0.5, end=0.5, size=0.05),  # You can adjust these parameters
        visible=True
    )
)

vis = visibility.copy()
vis[0] = True

dropdown_list.append(dict(
                args=[{"visible": vis}],
                label=teams[0],
                method="restyle"
            ))

for id, team in enumerate(teams[1:]):


# Plotly Contour Plot
    fig.add_trace(
        go.Contour(
            z=get_dens(team,df_2018),
            x=x,
            y=y,
            opacity = 0.6,
            colorscale='RdBu_r',
            contours_coloring='fill',
            contours=dict(start=-0.5, end=0.5, size=0.05),
            visible=False
        )
    )

    vis = visibility.copy()
    vis[id + 1] = True

    dropdown_list.append(dict(
                    args=[{"visible": vis}],
                    label=team,
                    method="restyle"
                ))


fig.update_layout(
    height=600,
    width=680,
    title = f'Season {2018}',
    title_font_size=30,
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_list,
            showactive=True,
            x=0.35,
            xanchor="left",
            y=1.18,
            yanchor="top"
        ),
    ]
)
fig.add_layout_image(
        dict(
            source=pyLogo,
            xref="x",
            yref="y",
            x=0,
            y=42.5,
            sizex=100,
            sizey=85,
            opacity=1,
            layer="below")
)

fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[0, 100])
fig.update_yaxes(
    showline=False,
    zeroline=False,
    showgrid=False,
    range=[-42.5, 42.5],
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

In [ ]:
from PIL import Image
nhl_2019 = NHLDataDownloader()

nhl_2019.set_season(2019)
df_2019 = nhl_2019.load_df_shots()
pyLogo = Image.open("figures/half_nhl_rink.png")

fig = go.Figure()

x = np.linspace(0, 89, 90)
y = np.linspace(-42, 42, 85)

teams = df_2019.Team.unique()
visibility = [False] * len(teams)
dropdown_list = []

fig.add_trace(
    go.Contour(
        z=get_dens(teams[0],df_2019),
        # z=np.flip(dens_team - dens_tot),
        x=x,
        y=y,
        opacity = 0.6,
        colorscale='RdBu_r',
        contours_coloring='fill',  # This will fill the contours
        contours=dict(start=-0.5, end=0.5, size=0.05),  # You can adjust these parameters
        visible=True
    )
)

vis = visibility.copy()
vis[0] = True

dropdown_list.append(dict(
                args=[{"visible": vis}],
                label=teams[0],
                method="restyle"
            ))

for id, team in enumerate(teams[1:]):


# Plotly Contour Plot
    fig.add_trace(
        go.Contour(
            z=get_dens(team,df_2019),
            x=x,
            y=y,
            opacity = 0.6,
            colorscale='RdBu_r',
            contours_coloring='fill',
            contours=dict(start=-0.5, end=0.5, size=0.05),
            visible=False
        )
    )

    vis = visibility.copy()
    vis[id + 1] = True

    dropdown_list.append(dict(
                    args=[{"visible": vis}],
                    label=team,
                    method="restyle"
                ))


fig.update_layout(
    height=600,
    width=680,
    title = f'Season {2019}',
    title_font_size=30,
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_list,
            showactive=True,
            x=0.35,
            xanchor="left",
            y=1.18,
            yanchor="top"
        ),
    ]
)
fig.add_layout_image(
        dict(
            source=pyLogo,
            xref="x",
            yref="y",
            x=0,
            y=42.5,
            sizex=100,
            sizey=85,
            opacity=1,
            layer="below")
)

fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[0, 100])
fig.update_yaxes(
    showline=False,
    zeroline=False,
    showgrid=False,
    range=[-42.5, 42.5],
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

In [ ]:
pyLogo = Image.open("figures/half_nhl_rink.png")

fig = go.Figure()

x = np.linspace(0, 89, 90)
y = np.linspace(-42, 42, 85)

teams = df_2020.Team.unique()
visibility = [False] * len(teams)
dropdown_list = []

fig.add_trace(
    go.Contour(
        z=get_dens(teams[0],df_2020),
        # z=np.flip(dens_team - dens_tot),
        x=x,
        y=y,
        opacity = 0.6,
        colorscale='RdBu_r',
        contours_coloring='fill',  # This will fill the contours
        contours=dict(start=-0.5, end=0.5, size=0.05),  # You can adjust these parameters
        visible=True
    )
)

vis = visibility.copy()
vis[0] = True

dropdown_list.append(dict(
                args=[{"visible": vis}],
                label=teams[0],
                method="restyle"
            ))

for id, team in enumerate(teams[1:]):


# Plotly Contour Plot
    fig.add_trace(
        go.Contour(
            z=get_dens(team,df_2020),
            x=x,
            y=y,
            opacity = 0.6,
            colorscale='RdBu_r',
            contours_coloring='fill',
            contours=dict(start=-0.5, end=0.5, size=0.05),
            visible=False
        )
    )

    vis = visibility.copy()
    vis[id + 1] = True

    dropdown_list.append(dict(
                    args=[{"visible": vis}],
                    label=team,
                    method="restyle"
                ))


fig.update_layout(
    height=600,
    width=680,
    title = f'Season {2020}',
    title_font_size=30,
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_list,
            showactive=True,
            x=0.35,
            xanchor="left",
            y=1.18,
            yanchor="top"
        ),
    ]
)
fig.add_layout_image(
        dict(
            source=pyLogo,
            xref="x",
            yref="y",
            x=0,
            y=42.5,
            sizex=100,
            sizey=85,
            opacity=1,
            layer="below")
)

fig.update_xaxes(showline=False, zeroline=False, showgrid=False, range=[0, 100])
fig.update_yaxes(
    showline=False,
    zeroline=False,
    showgrid=False,
    range=[-42.5, 42.5],
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()